In [ ]:
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import six
from pyspark.sql.functions import isnan,when,count,col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [ ]:
Boston = pd.read_csv("C:/Users/hp/Downloads/boston.csv")

In [ ]:
Boston.head()

In [ ]:
spark = SparkSession.builder \
.master("local").appName("hive_pyspark").enableHiveSupport().getOrCreate()

In [ ]:
data_boston=spark.read.csv('C:/Users/hp/Downloads/boston.csv',header=True,inferSchema=True)

In [ ]:
data_boston.describe().show()

In [ ]:
data_boston.select([count(when(isnan(c),c)).alias(c) for c in data_boston.columns]).show()

In [ ]:
for i in data_boston.columns:
    if not( isinstance(data_boston.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, data_boston.stat.corr('MV',i))

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')
new_data= vectorAssembler.transform(data_boston)
new_data = new_data.select(['features', 'MV'])
new_data.show()

In [ ]:
splits = new_data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

In [ ]:
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

In [ ]:
training = lr_model.summary
print("RMSE: %f" % training.rootMeanSquaredError)
print("r2: %f" % training.r2)